# Produce Streaming Data

Before you run the following script, launch the Docker container with this command from the Terminal:

`cd real-time`

`docker-compose up -d`

In [3]:
# Part 1: Application Setup --
from quixstreams import Application
import csv
import json
import time

# Create an Application - the main configuration entry point
app = Application(broker_address="localhost:9092", consumer_group="demand_forecasting")

# Define a single topic for both types of data
combined_data_topic = app.topic(name="incoming_data", value_serializer="json")


# Part 2: Simulating Streaming Data --

# Paths to the CSV files with data
ENERGY_DATA_FILE = "data/energy_data.csv"
TEMPERATURE_DATA_FILE = "data/weather_data.csv"

def load_csv(path: str):
    rows = []
    with open(path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Convert all string fields that should be numeric
            for key, value in row.items():
                try:
                    row[key] = float(value)
                except ValueError:
                    pass
            rows.append(row)
    return rows

# Load data from the CSV files
energy_data = load_csv(ENERGY_DATA_FILE)
temperature_data = load_csv(TEMPERATURE_DATA_FILE)

print(f'Writing CSV data to the "{combined_data_topic.name}" topic ...')
with app.get_producer() as producer:
    # Determine the length of the shorter dataset to avoid index errors
    min_length = min(len(energy_data), len(temperature_data))
    
    for i in range(min_length):
        # Access the current row from energy data
        energy = energy_data[i]
        # Prepare a message for energy data
        energy_message = {
            "Period": energy['period'],
            "Type": "energy",
            "Value": energy['value']
        }
        
        # Send the energy data message to the combined topic
        producer.produce(
            topic=combined_data_topic.name,
            key="energy-csv",
            value=json.dumps(energy_message),
        )
        
        time.sleep(2) # Simulate real-time flow with a sleep for energy data
        
        # Access the current row from temperature data
        temp = temperature_data[i]
        # Prepare a message for temperature data
        temperature_message = {
            "Period": temp['period'],
            "Type": "temperature",
            "Value": temp['temperature']
        }
        
        # Send the temperature data message to the same combined topic
        producer.produce(
            topic=combined_data_topic.name,
            key="temperature-csv",
            value=json.dumps(temperature_message),
        )
        
        time.sleep(3)  # Simulate real-time flow with a sleep for temperature data

# Part 3: Shutting Down ---
# Ensure all messages are sent before closing
producer.flush()

[2024-10-07 13:19:21,372] [INFO] [quixstreams] : Topics required for this application: "incoming_data"
[2024-10-07 13:19:21,405] [INFO] [quixstreams] : Creating a new topic "incoming_data" with config: "{'num_partitions': 1, 'replication_factor': 1, 'extra_config': {}}"


Writing CSV data to the "incoming_data" topic ...


[2024-10-07 13:19:22,409] [INFO] [quixstreams] : Topic "incoming_data" has been created
[2024-10-07 13:19:22,410] [INFO] [quixstreams] : Validating Kafka topics exist and are configured correctly...
[2024-10-07 13:19:22,436] [INFO] [quixstreams] : Kafka topics validation complete
[2024-10-07 13:23:14,613] [INFO] [quixstreams] : FAIL [rdkafka#producer-10] [thrd:localhost:9092/bootstrap]: localhost:9092/1: Disconnected (after 231514ms in state UP)
[2024-10-07 13:23:14,615] [INFO] [quixstreams] : FAIL [rdkafka#producer-10] [thrd:localhost:9092/bootstrap]: localhost:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
[2024-10-07 13:23:14,616] [INFO] [quixstreams] : FAIL [rdkafka#producer-10] [thrd:localhost:9092/bootstrap]: localhost:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrec

KeyboardInterrupt: 